Exercise 1. Implement integration over a rectangular domain with a double Legendre quadrature.

In [1]:
# импорт необходимых библиотек
from scipy.integrate import dblquad, tplquad, nquad, quadrature
import numpy as np

In [2]:
# задаем функцию, интеграл от которой будем брать, а также концы интегрируемых отрезков и число членов в суммах
f = lambda x, y: x**2 + 3*y**3
a1, a2 = 0, 0
b1, b2 = 3, 3
n, m = 1000, 1000

In [3]:
# функция, выполняющая интегрирование по алгоритму Лежандра-Гаусса
def integration_over_rectangular_double_Legendre(f,a1=0,b1=1,a2=0,b2=1,n=1000,m=1000):
    integral = 0 
    for i in range(0, n):
        x = a1 + (b1 - a1) * (i + 0.5) / n
        for j in range(0, m):      
            y = a2 + (b2 - a2) * (j + 0.5) / m
            integral += f(x, y) * (b2 - a2)*(b1 - a1) / (n * m)
    return(integral)

In [4]:
# функция, проверяющая корректность подсчета интеграла
def check_correct():
    calc = integration_over_rectangular_double_Legendre(f, a1, b1, a2, b2, n, m)
    exact = dblquad(f, 0, 3, lambda x: 0, lambda x: 3)[0]
    print('Exact:', exact, '\n', 'Calculate:', calc)

In [5]:
# вызов функции
check_correct()

Exact: 209.25 
 Calculate: 209.24990212501024


#### Как мы видим, наш подсчет довольно точный.

Exercise 2. Implement integration over a trangular domain with a mixed Legendre/Jacobi quadrature. 

In [6]:
# вспомогательная функция подсчета якобиана преобразования, отображающего треугольную область в единич квадрат
def dJ( u, v, p1, p2, p3 ):
    
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    dxdu = ( (1 - v) * x2 + v * x3 - x1 )
    dxdv = ( u * x3 - u * x2 )
    dydu = ( (1 - v) * y2 + v * y3 - y1 )
    dydv = ( u * y3 - u * y2 )
    return np.abs( dxdu * dydv - dxdv * dydu )

In [7]:
# функция двойного квадратурного интегрирования по треугольной области
# на входе - интегрируемая функция и вершины треугольника
# на выходе - посчитанный интеграл и абсолютная ошибка подсчета
def tridblquad( integrand, p1, p2, p3 ):
    
    x1, y1 = p1 ; x2, y2 = p2 ; x3, y3 = p3
    # функция-преобразование в единичный квадрат
    g = lambda u, v, c1, c2, c3: (1-u)*c1 + u*( (1-v)*c2 + v*c3 )
    # преобразуем наш интеграл с учетом якобиана-множителя
    def h( u, v ):
        x = g( u, v, x1, x2, x3 )
        y = g( u, v, y1, y2, y3 )
        I = integrand( x, y )
        I *= dJ( u, v, p1, p2, p3 )
        return I
    # считаем интеграл известным пакетом в новом пространстве
    integral, error = dblquad( h, 0, 1, lambda x: 0, lambda x: 1)
    return integral, error

In [8]:
# Проверка проводится на подсчете площади треугольника с вершинами (0,0), (2,0), (0,2)
# Площадь такого треугольника, очевидно, должна равняться 2
area, error = tridblquad( lambda x, y: 1., (0.,0.), (2.,0.), (0.,2.) )
print ('Calculate:',area)

Calculate: 2.0
